## OpenCV를 이용한 신호등 검출 

### BLOB 파라미터 설정 

In [10]:
def set_blob_param(r_type: str):
    # BLOB 필터 생성하기 
    params = cv2.SimpleBlobDetector_Params()
    
    params.minThreshold = 10
    params.maxThreshold = 255
    params.thresholdStep = 3
    params.filterByArea = False
    params.filterByColor = False
    params.filterByCircularity = True
    params.filterByConvexity = False 
    params.filterByInertia = False 
    
    if r_type == 'color':
        # 경계 값 조정
        params.minCircularity = 0.8
        params.maxCircularity = 0.85     
    elif r_type == 'shape':
        params.filterByArea = True 
        params.minArea = 100
        params.maxArea = 250
        
        params.filterByColor = False
        params.minCircularity = 0.4
        params.maxCircularity = 0.8
    
    return params 

### 신호등으로 추정되는 영역(shape) 검출 

In [17]:
def subtract_shape(img):
    global k, h, w   # 모폴로지 연산 필터 
    global blk_size, C 
    global data_output_path
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(data_output_path + 'img_gray.png',img_gray)

    roi_s = img_gray[int((1/5)*w):int((3/5) * w), int((1/5)*h): int((3/5)*h)]
    
    blur = cv2.GaussianBlur(roi_s, (5, 5), 0)
    cv2.imwrite(data_output_path + 'blur.png', blur)
    
#     cv2.imshow('blur', blur)
    _, th = cv2.threshold(roi_s, 55, 255, cv2.THRESH_BINARY)
    _, th2 = cv2.threshold(blur, 55, 255, cv2.THRESH_BINARY)
    _, t_otsu = cv2.threshold(blur, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    th_mean = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blk_size, C)
    th_gaussian = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blk_size, C)
    cv2.imwrite(data_output_path + 'th.png',th)
    cv2.imwrite(data_output_path + 't_otsu.png',t_otsu)
    cv2.imwrite(data_output_path + 'th_mean.png', th_mean)
    cv2.imwrite(data_output_path + 'th_gaussian.png', th_gaussian)
    
#     cv2.imshow('th', th) 
#     cv2.imshow('edge_s', edge_s)
    dst_s = cv2.morphologyEx(t_otsu, cv2.MORPH_OPEN, k)   # 모폴로지 열기 |연산 
    cv2.imwrite(data_output_path + 'morp_o.png',dst_s)
    edge_s = cv2.Laplacian(dst_s, -2)
    cv2.imwrite(data_output_path + 'laplacian.png',edge_s)
#     edge_s = cv2.Canny(img, 100, 200)
#     cv2.imshow('dst', dst_s)
    detector = cv2.SimpleBlobDetector_create(param_s)
    
    s_keypoints = detector.detect(edge_s)
    img_draw_s = cv2.drawKeypoints(roi_s, s_keypoints, None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
    cv2.imwrite(data_output_path + 'img_draw.png',img_draw_s)
    
    global shape_list 
    
    for s_keypoint in s_keypoints:
        roi_cx, roi_cy = map(int, s_keypoint.pt)
        cx, cy = int(roi_cx + h * (1/5)), int(roi_cy + w * (1/5))
        cv2.putText(img_gray, '!', (cx, cy), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))
        size = int(s_keypoint.size)
        cx2, cy2 = cx + size // 2, cy + size // 2
#         cx2, cy2 = cx + size, cy + size 
#         cv2.rectangle(img_gray, (cx, cy), (cx2, cy2), (255, 0, 0))
        shape_list.append([cx, cx2, cy, cy2])
    
    cv2.imshow('img_gray', img_gray)     
    cv2.imwrite('img_gray2.png', img_gray)
#     print(len(s_keypoints))
    return shape_list 

### 신호등 색상에 해당하는 영역 검출 

In [23]:
def subtract_color(img, color):
    global k, w, h   # 모폴로지 연산 필터 
    global data_output_path
    
    lower_green = np.array([50, 50, 80]); upper_green = np.array([90, 255, 255])
    lower_red = np.array([-10, 30, 50]);  upper_red = np.array([10, 255, 255])
    lower_yellow = np.array([11, 50, 50]); upper_yellow = np.array([30, 200, 200])

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imwrite(data_output_path + 'img_hsv.png',img_hsv)
    cv2.imshow('img_hsv', img_hsv)
    roi_c = img_hsv[int((1/5)*w):int((3/5) * w), int((1/5)*h): int((3/5)*h)]
    
    if color == 'red':
        mask = cv2.inRange(roi_c, lower_red, upper_red)
    elif color == 'green':
        mask = cv2.inRange(roi_c, lower_green, upper_green)   
    elif color == 'yellow':
        mask = cv2.inRange(roi_c, lower_yellow, upper_yellow) 
    # cv2.imshow('mask', mask)
    img_mask = cv2.bitwise_and(roi_c, roi_c, mask=mask)
    cv2.imwrite(data_output_path + 'img_mask.png',img_mask)
    edge_c = cv2.Laplacian(img_mask, -2)
#     cv2.imshow('Laplacian', edge_c)
    cv2.imwrite(data_output_path + 'img_lap_c.png',edge_c)
#     dst_c = cv2.morphologyEx(edge_c, cv2.MORPH_OPEN, k)   # 모폴로지 닫기 연산 
    # cv2.imshow('dst', dst)

    detector = cv2.SimpleBlobDetector_create(param_c)
    c_keypoints = detector.detect(edge_c)
    img_draw_c = cv2.drawKeypoints(roi_c, c_keypoints, None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(data_output_path + 'roi_c.png',roi_c)
    
    global color_list 
    
    for c_keypoint in c_keypoints:
        roi_cx, roi_cy = map(int, c_keypoint.pt)
        cx, cy = int(roi_cx + h * (1/5)), int(roi_cy + w * (1/5))
        cv2.putText(img_hsv, '!', (cx, cy), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))
        size = int(c_keypoint.size)
        cx2, cy2 = cx + size // 2, cy + size // 2 
#         c2, cy2 = cx + size, cy + size 
        color_list.append([cx, cx2, cy, cy2])
#     cv2.imshow('roi_c', roi_c)
    cv2.imshow('img_hsv2', img_hsv)    
    cv2.imwrite('img_hsv2.png', img_hsv)
#     print(color_list)
    return color_list

### 겹치는 구간 판단

In [24]:
def check_location(img, shape_list, color_list):   # 직사각형 겹치는 좌표 개수 구하기 
    global tl_flag, color_flag
    for a1, a2, b1, b2 in color_list:
        for x1, x2, y1, y2 in shape_list:
            if ((a1 >= x1 and a1 <= x2) and (b1 >= y1 and b1 <= y2)) and tl_flag == False:
                cv2.putText(img, f'{color_flag} light Detected !', (a1, b1), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))
                cv2.imwrite(data_output_path + 'light_detected.png',img)
                tl_flag = True 

### 실행 

In [31]:
import cv2
import numpy as np 

video_file = './cropped_traffic2.mp4'
cap = cv2.VideoCapture(video_file)
data_output_path = './result/'

w = 400; h = 400  # roi 지정 
blk_size = 9; C = 5  # 적응형 스레시 홀드 필터 
k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))   # 모폴로지 연산 필터 
color_flag = 'green'   # 검출하고 싶은 색상 입력 

if cap.isOpened():
    fps = cap.get(cv2.CAP_PROP_FPS) 
    delay = int(1000/fps)
    
    param_c = set_blob_param('color')
    param_s = set_blob_param('shape')
    
    shape_list = []
    color_list = [] 
    
    while True:
        ret, img = cap.read()
        if ret:          
#             cv2.imwrite(data_output_path + 'img.png', img)
            res_c = cv2.resize(img, (400, 400))
            shape_list = subtract_shape(res_c)
            color_list = subtract_color(res_c, color_flag)
            
            check_location(res_c, shape_list, color_list)   # 1차 수정 
            cv2.imshow('img', res_c)
            tl_flag = False;
            if cv2.waitKey(1) != -1:
                break 
#             cv2.waitKey(delay)
        else:
            break
    
    print(len(shape_list))
    print(len(color_list))
else:
    print("can't open video.")

cap.release()
cv2.destroyAllWindows()

311
509
